#TikTakToe
The famous game solved using some famous machine-learning algorithms:
* MiniMax
* NeuralNetwork

The MiniMax algorithm consists in generating a tree of the possible outcomes of a run. Two players alternate moves, and each set of moves can be translated in a node of the tree.
The tree is explored up to the leafs, which are the end states of the game. Considering which player has won at the end, each final leaf is associated with a +1 if the first player won, -1 if the second player won, and 0 if no one is winning.
The values are backpropagated up to the root. When the state/node and the player are determined, the algorithm allow to select a winning branch.



Convention: the TicTacToe board is composed by 9 cells. The configuration remember the layout of the numeric keypads. Therefore, as convention 1 is the top-left cell, followed by 2 and 3 on its right. 4 is below 1, and 9 is at the bottom right of the board.


Below, the class Node, representing each state of the game, and the tree are initialized. During initialization, the tree values are not initialized, except for the leafs. Each leafs is associated with a value +1 or -1, based on the winning player.

In [1]:
class Node(object):

    def __init__(self,name=0, path =[], link=[], value = 100,player= 1,blcklist = [1,2,3,4,5,6,7,8,9],
                                                                                         wthlist=[], rlist=[],blist=[]):
        self.name= name               # numerical ID of the node
        self.path = path              # numerical ID sequences, how we have reached this node from the original root node
        self.link = link              # ID of sons nodes
        self.value = value
        self.player = player
        self.blackList = blcklist     # available moves from the current state
        self.whiteList = wthlist      # collect the moves to reach the node
        self.redList = rlist          # moves of the player red
        self.blueList = blist         # moves of player blue

In [ ]:
import copy
import numpy as np

# this utility function first retrieve elements not common to 2 lists
# then return the first element
def get_move(list1,list2):
    s = [i for i in list1 if i not in list2]
    return s[0]

#-------------------------------------------------------------------------------
# this function will be useful to determine winning state
# the input in an array of numbers, corresponding to the positions
# of the TicTacToe cells on a board
def check_winner(the_list):
    greenList = [ [1,2,3], [4,5,6], [7,8,9],[1,4,7], [2,5,8], [3,6,9],[1,5,9], [3,5,7]]
    answer = False
    for el in greenList:
        if el[0] in the_list and el[1] in the_list and el[2] in the_list:
            return True
    return answer
#-------------------------------------------------------------------------------

listOfNodes = [ [] , [], [], [], [] ,[] ,[] ,[] ,[] ,[] ]  # this variable will store nodes subdivided by layer ( 0-> init game, 10-> end of the game)
keys = dict()                                              # this variable will be useful to store nodes name as key and values (-1,0,1) as values
#--------------
node0 = Node(name =0, path=[0], player =0)    # first node creation ==> empty node
listOfNodes[0].append(node0)
#--------------
counter =0     # used to give name to new nodes
#--------------
# main cycle -  tree creation

for i in range(len(listOfNodes)):   # 1-> 9
    subArray = listOfNodes[i]

    for h in range(len(subArray)):
        node = subArray[h]

        if node.value == 100:    # is the tree already ended? if not, go ahead  (100 is a default value != -1,0,1)
            #--------------
            # from the array of elements, delete one item at time to have all combinations
            list_of_black = []                        # very ugly but there were some problems...
            for bl in range(1,len(node.blackList)+1): # it creates an array for many new black lists
                x= copy.copy(node.blackList)
                del x[bl-1]
                list_of_black.append(x)

            #--------------
            for kk in range(len(list_of_black)):      # now we create parameters for new nodes
                new_black = copy.copy(list_of_black[kk])     # an array
                mossa = get_move(node.blackList,new_black)   # the difference is the move done
                #--------------
                counter +=  1
                new_name = counter
                #--------------
                new_path = copy.copy(node.path)
                new_path.append(counter)
                #--------------
                new_white = copy.copy(node.whiteList)
                new_white.append(mossa)
                #--------------
                new_value = 100     # we don't know if a winning node... in case we put different values after

                # who is the actual player for that node
                player = copy.copy(node.player)
                if player == -1 or player == 0:
                    new_player = 1
                    new_redlist = copy.copy(node.redList)
                    new_redlist.append(mossa)
                    #--------------
                    # check if there is winner
                    answer = check_winner(new_redlist)
                    if answer== True:
                        new_value = 1
                        keys[counter] = 1
                    new_bluelist = copy.copy(node.blueList)

                else:
                    new_player = -1
                    new_bluelist = copy.copy(node.blueList)
                    new_bluelist.append(mossa)
                    #--------------
                    # check if there is winner
                    answer = check_winner(new_bluelist)
                    if answer== True:
                        new_value =-1
                        keys[counter] = -1     # <== the dictionary   ['node_number': 'value']
                    new_redlist = copy.copy(node.redList)

                # we don't have the winner, moves are ended
                if  len(new_black )==0 and answer==False:
                    new_value =0
                    keys[counter] = 0

                # finally create the node with new numbers calculated
                new_node = Node(name = new_name,path=new_path,link=[],value = new_value,
                                player= new_player,blcklist =new_black, wthlist=new_white,
                                                    rlist=new_redlist,blist=new_bluelist )
                node.link.append(counter)   # update 'old' node with the son node
                listOfNodes[i+1].append(new_node)

print("tree initialized")
# at the conclusion we have our tree with all values =100 except for final nodes

tree initialized


Now that the tree is built, we need a decision-rule. This rule consists in deciding which move select, aka which branch we have to follow.


Backpropagating the values from the leafs, we can associate a winning branch for all the nodes.
In particular, if a node belongs to "max" player, we will select among the "sons nodes" the one which correspond to the maximum value (+1).
On the contrary, if this node belongs to "min" player, we will select the node among "sons nodes" which corresponds to the minimum value (-1).

Below, starting from the leafs, the value are backpropagated to the root.

In [ ]:
def find_Value( value_list, player_coeff):
    # from a list of values (-1,0,1), retrieve the maximum value
    # considering the "color" of the player, -1 or 1
    if player_coeff < 0:
      min_val = min(value_list)
      min_pos = value_list.index(min_val)
      return [min_val, min_pos]
    else:
      max_val = max(value_list)
      max_pos = value_list.index(max_val)
      return [max_val, max_pos]



#------------------------
# MINIMAX ALGO
indexes = [n for n in range(0,10)] [::-1]    # start from the last layer

for i in indexes:
  print("updating layer ",i,"/9")
  subArray = listOfNodes[i]   # nodes in the right-most

  for n in range(len(subArray)):
      node = subArray[n]      # a node
      #------------------------
      if len(node.link)>0 :  # if it has sons
          player = copy.copy(node.player)
          if player ==0:
              player = 1

          # now create for the node array for link and values of corresponding nodes
          sons = []
          values_of_sons=[]

          for l in range(len(node.link)):
              sons.append(node.link[l])
              son_value = keys.get(node.link[l]) # the value gathered from the leafs
              values_of_sons.append(son_value)

          values = find_Value(values_of_sons,(-1)*player)
          node.value = values[0]
          node.link =[sons[values[1]]]
          keys[node.path[-1]] = values[0]  # update the value of this node
print("tree updated for all players")

Now we can play.   
The player will have first to select if it is starting, or if "machine" will move fist.   
The game accepts numerical inputs in the range 1-9, which corresponds to the cells of the board.

In [ ]:
# now we can start with the game

def find_node(subArray, WhiteList):
    for i in range(len(subArray)):
        node = subArray[i]

        if node.whiteList== WhiteList:
            return i
    return -1

def next_node(subArray, Name):
    for i in range(len(subArray)):
        node = subArray[i]
        if node.name == Name:
            return node
    return False


#-------------------------
def human_turn(whiteList):
    #n= n+1
    human = int(input("give your next move: "))
    if human in whiteList:
        if len(whiteList)<1:
            #print("game ended")
            return False
        else:
            print("invalid key")
        return False
    else:
        next_move = human
    return next_move
#---------------------------
def machine_turn(n,listOfNodes, whiteList):
    subArray = copy.copy(listOfNodes[n])
    position = find_node(subArray, whiteList)
    if position == -1:
        return False
    else:
        node = subArray[position]
        if len(node.link)>0:
            next_link = node.link[0]
            new_node_name = next_link
            new_node = next_node(listOfNodes[n+1], new_node_name)
            next_move = get_move(node.blackList,new_node.blackList)
            return next_move
        #else:
        #    print("game ended")
    return True


#--------------------------

def printBoard(board):
    print("\n")
    print(board["1"] + ' | ' + board["2"] + ' | ' + board["3"])
    print('-- + -- + --')
    print(board["4"] + ' | ' + board["5"] + ' | ' + board["6"])
    print('-- + -- + --')
    print(board["7"] + ' | ' + board["8"] + ' | ' + board["9"])

#--------------------------

def Game(starting_machine, listOfNodes):
    board = {"7": '  ' , "8": '  ' , "9": '  ' ,"4": '  ' , "5": '  ' , "6": '  ' ,"1": '  ' , "2": '  ' , "3": '  ' }

    whiteList = []
    moves_human = [  ]
    moves_machine = [  ]
    #-------------------------
    if starting_machine == "no":
        n = -1
        for j in range(0,5):

            n = n+1
            next_move = human_turn(whiteList)
            if next_move == False:
                return False
            whiteList.append(next_move)
            board[str(next_move)] = " X"
            moves_human.append(next_move)

            if check_winner(moves_human):
              print("game ended  - human winner !")
              printBoard(board)
              return True
            printBoard(board)
            #---------------------------
            n = n+1
            next_move = machine_turn(n,listOfNodes, whiteList)
            whiteList.append(next_move)
            board[str(next_move)] = " O"
            moves_machine.append(next_move)
            printBoard(board)

            if check_winner(moves_machine):
              print("game ended  - machine winner !")
              return True

        return True
    #-------------------------
    else:
        n = -1
        for j in range(0,5):
            n = n+1
            next_move = machine_turn(n,listOfNodes, whiteList)
            whiteList.append(next_move)
            board[str(next_move)] = " O"
            moves_machine.append(next_move)
            printBoard(board)

            if check_winner(moves_machine):
              print("game ended  - machine winner !")
              return True
            #---------------------------
            n = n+1
            next_move = human_turn(whiteList)
            whiteList.append(next_move)
            board[str(next_move)] = " X"
            moves_human.append(next_move)

            if check_winner(moves_human):
              print("game ended  - human winner !")
              printBoard(board)
              return True
        return True
    print("\n")

#machine = bool(input("is machine starting? --   \nTrue if yes \nFalse if not\n"))
machine =  input("is machine starting? --   \nyes \nno\n")
print("you are 'X', machine is 'O'")
Game(machine,listOfNodes)

Neural Networks are wide used in a vast range of different problems. As well, Neural Networks are used in Video Games to enable autonomous reasoning.   

The next session will use the famous Pytorch library (https://pytorch.org/) to train a Multi Layer Perceptron able to play the game.
The first problem is how we get the training data. Once we have our MiniMax algorithm, we let a MiniMax agent play against another automatic agent to generate a dataset. Each entry of the dataset has as many features as the number of cells of the board. Each feature corresponds to a cell, and its value is:  
* +1 if the machine1 made the move in that cell  
* -1 if the machine2 made the move in that cell
* 0 if it is still available

The targets are the desired output of our network. The targets are coded in the same way as the feature, but the values are:
* 1 if that cell is our desired move
* 0 othervise

once we can associate a board-state before the agent's move and its corresponding best move, we can train the network.   

The following snippet is a battle between two machines to collect data. Because we need to explore the solution space, one machine follows the exact MiniMax algo, while the other is subject to randomness. This is like a Monte Carlo Simulation.

In [ ]:
import random
import itertools


# generate dataset for the the ANN training
starting_machine =1
number_rounds = 1000
random_probability = 0.85


def generate_dataset(number_rounds, random_probability):
  dataset = []    # each entry will be 9 position for board state + 9 final number marking desired position as one -hot encoding
                  # to be returned
  # to print advance
  for round in range(number_rounds):
    if round%100 == 0:
      print("round: " + str(round))

    # play different games machine vs machine
    whiteList = []
    moves_machine1 = [  ]
    moves_machine2 = [  ]
    all_moves = [ i for i in range(1, 10)]
    board_state = [0, 0, 0, 0, 0, 0, 0, 0, 0]

    #print( "new game" )
    n = -1
    for j in range(0,5):

      n = n+1
      # machine 2 sometimes do wrong
      rand = random.uniform(0, 1)
      if ( rand < random_probability):
        available_moves = [x for x in all_moves if x not in whiteList]
        # random move
        next_move = random.choice(available_moves)
      else:
        next_move = machine_turn(n,listOfNodes, whiteList)
      #print(next_move)
      whiteList.append(next_move)
      moves_machine2.append(next_move)
      board_state[next_move-1] = -1
      #print(board_state )
      if check_winner(moves_machine2) == True:
          j = 1000
          continue
      elif len(whiteList) >= 9:
          j = 1000
          continue

      n = n+1
      # machine 1 makes best move always
      next_move = machine_turn(n,listOfNodes, whiteList)
      whiteList.append(next_move)
      moves_machine1.append(next_move)

      new_data = [ b for b in board_state]  # board before new move
      target = [ 0 for _ in range(1,10)]
      target[next_move-1] = 1
      dataset.append( new_data + target)

      board_state[next_move-1] = 1           # new board state

      #print(board_state )
      if check_winner(moves_machine1) == True:
          j = 1000
          break
      elif len(whiteList) >= 9:
          j = 1000
          break
  return dataset

# a dataset with all the generated runs and outcomes
dataset = generate_dataset(number_rounds, random_probability)

# remove duplicates, to not bias the training that will follow
dataset.sort()
clean_dataset = list(dataset for dataset,_ in itertools.groupby(dataset))   # remove duplicate

# based on currently available outcomes, because the two machines have been playing always
# in the same side, we generate a specular array as if the two machine would have swapped side
specular = []
maxLen = len(clean_dataset)
for i in range(maxLen):
  data_entry = clean_dataset[i]
  features = [ -1*x for x in data_entry[0:9]]
  targs = [ t for t in data_entry[9:18] ]
  specular_array = features + targs
  clean_dataset.append(specular_array)


print("dataset size: " + str(len(clean_dataset)))

round: 0
round: 100
round: 200
round: 300
round: 400
round: 500
round: 600
round: 700
round: 800
round: 900
dataset size: 804


Then, we initialize and train the Neural Network.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import torch.nn.functional as F
import math
from collections import deque
import matplotlib.pyplot as plt


class ANN(nn.Module):
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        self.linear1 = nn.Linear( in_size, hidden_size)
        self.linear2 = nn.Linear( hidden_size, hidden_size)
        self.outLayer = nn.Linear( hidden_size, out_size)
        self._init_weights()

    def forward(self, x):
          x = torch.relu(self.linear1(x))
          x = torch.relu(self.linear2(x))
          x = torch.sigmoid(self.outLayer(x))
          return x

    def _init_weights(self):
        # initialize weights in proper way
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)


# initialize the features and the targets(labels)
clean = np.array( [  np.array(cl) for cl in clean_dataset])

only_features = clean[:,0:9]
only_targets = clean[:,9:18]
targets = torch.tensor(only_targets, dtype=torch.float32)
features = torch.tensor(only_features, dtype=torch.float32)

model = ANN(in_size= 9, hidden_size=64, out_size= 9 )
print(model)


# loss and optimization
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# batch handler
loader = DataLoader(TensorDataset(features,targets), batch_size=10, shuffle=True)


# training loop
epoch_number = 300
for epoch in range(epoch_number):

  for X_batch, y_batch in loader:
    optimizer.zero_grad()
    outputs = model(X_batch)
    loss = criterion(outputs, y_batch)
    loss.backward()
    optimizer.step()

model.eval()

ANN(
  (linear1): Linear(in_features=9, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=64, bias=True)
  (outLayer): Linear(in_features=64, out_features=9, bias=True)
)


'\ncount = 0\nfor d in range(len(features) ):\n  prediction = model.forward(features[d])\n\n  max_pred = max(prediction)\n  b = prediction == max_pred\n  max_pos_pred = b.nonzero()\n\n\n  max_tar = max(targets[d])\n  c = targets[d] == max_tar\n  max_pos_tar = c.nonzero()\n\n  if max_pos_tar == max_pos_pred:\n    count +=1\n\nprint(count/ len(features))\n'


Now we can play. Our agent should be trained at least to defeat us...

In [ ]:
def play_vs_ANN(network):

    # retrieve the ANN's move, that must be feasible
    def decide_move( wlist, network_outputs):
      for w in reversed(wlist):
        network_outputs[w-1] = -100000000000      # cannot be chosen

      maxval = max(network_outputs )
      max_pos = network_outputs.index(maxval)
      return max_pos+1
    # ----------------------------------------------

    board = {"7": '  ' , "8": '  ' , "9": '  ' ,"4": '  ' , "5": '  ' , "6": '  ' ,"1": '  ' , "2": '  ' , "3": '  ' }

    whiteList = []
    moves_human = []
    moves_machine = []
    board_state = [0, 0, 0, 0, 0, 0, 0, 0, 0]

    n = -1
    for j in range(0,5):
        n = n+1

        outputs = network.forward(torch.tensor(board_state, dtype=torch.float32))

        next_move = decide_move(whiteList, outputs.tolist() )
        whiteList.append(next_move)
        board_state[next_move-1] = 1

        board[str(next_move)] = " O"
        moves_machine.append(next_move)
        printBoard(board)

        if check_winner(moves_machine):
          print("game ended  - machine winner !")
          return True
        #---------------------------
        n = n+1
        next_move = human_turn(whiteList)
        whiteList.append(next_move)
        board_state[next_move-1] = -1

        board[str(next_move)] = " X"
        moves_human.append(next_move)

        if check_winner(moves_human):
          print("game ended  - human winner !")
          printBoard(board)
          return True

        if len(whiteList) >= 9:
          print("no winner")
          return True

play_vs_ANN(model)


A Neural Network can also be trained to approximate the Q-table. In reinforcement learning, the Q-table is used by autonomous agents to take decisions. The table correlate a state, such as a board state in the tic-tac-toe game, and an action, such as what is the next move.    
A Neural Network can be used in place of the table.  

Next, a new Neural Network is defined.

In [ ]:

class QNN(nn.Module):
    def __init__(self, in_size=9, hidden_size=128, out_size=9):
        super().__init__()
        self.layer1 = nn.Linear(in_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.layer3 = nn.Linear(hidden_size, hidden_size)
        self.layer4 = nn.Linear(hidden_size, out_size)
        #self.activation = nn.LeakyReLU(0.01)

    def forward(self, x):
        x = F.leaky_relu(self.layer1(x))
        x = F.leaky_relu(self.layer2(x))
        x = F.leaky_relu(self.layer3(x))
        #x = self.activation(self.layer1(x))
        #x = self.activation(self.layer2(x))
        x = self.layer4(x)  # nessuna attivazione qui
        return x


To train the Network, the agent is initialized with a default model. The agent can decide, but it is not trained to play effectively.  
Training requires "experience", such as games in which the agent collects rewards (if wins) or penalties (if looses).  
A proper training is structured to offer a good set of experience to the agent, also know as exploration vs esploration trade-off.   
In the code below, esperiences are collected playing the game, and stored in a replayBuffer object. Time by time, the agent look back in ints memory and updates its status.

In [ ]:

# data format to generate a state of this run
def generate_memory_entry(board_state, next_move, reward, is_end_game, one ):    # next_move già normalizzato 0 - 8
        new_memory = np.array( [0 for _ in range(21)] )
        new_memory[0:9] = np.array(board_state)     # current state of the board
        new_memory[9:18] = np.array(board_state)    # this line and the one below  update the board to next state
        new_memory[9 + next_move] = one             # 1 or -1 on the basis of the player
        new_memory[18] = next_move          # move coded as  0 - 8
        new_memory[19] = reward
        new_memory[20] = is_end_game        # only if win, loose or draw
        return new_memory


def Qtraining( Qmodel, number_rounds, greedy_probability):

  # data format to store states of the game
  memory_states = np.zeros(shape=(1, 21))

  # rewards if win, loss, nothing happens or draw
  r_win =  10
  r_loss =  -1
  r_null = -0.01
  r_draw = 0.01

  for round in range(number_rounds):   # play the game

    whiteList = []
    moves_machine1 = [  ]
    moves_machine2 = [  ]
    all_moves = [ i for i in range(1, 10)]
    board_state = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    # ---

    conclude_experience = 0   # 1 if run must end
    reward = 0                # according if win, lost or invalid move
    n = -1
    for j in range(0,5):

      n = n+1
      # ------------------------------------------------------------------------
      # machine 1 is our agent to be trained  ----------------------------------
      rand = random.uniform(0, 1)
      if ( rand < greedy_probability):    # explore new solutions
        available_moves = [x for x in all_moves if x not in whiteList]
        next_move = random.choice(available_moves) -1
      else:                               # choose action with the network
        out = Qmodel.forward( torch.tensor( np.array(board_state)+2 , dtype=torch.float32) )
        next_move = torch.argmax(out)     # 0 - 8

      # check if it is a valid move
      if next_move in  whiteList:
          conclude_experience = 1
          reward = r_loss
          new_memory = generate_memory_entry(board_state, next_move, reward, conclude_experience,1 ) # save the status
      else:
          whiteList.append(next_move+1)
          moves_machine1.append(next_move+1)

          if check_winner(moves_machine1) == True:
            conclude_experience = 1
            reward = r_win
            new_memory = generate_memory_entry(board_state, next_move, reward, conclude_experience,1 )
          elif len(whiteList) >= 9:
            conclude_experience = 1
            reward = r_draw
            new_memory = generate_memory_entry(board_state, next_move, reward, conclude_experience,1 )

          # save an intermediate state
          if conclude_experience ==0:
            reward = r_null
            new_memory = generate_memory_entry(board_state, next_move, reward, conclude_experience,1 )


          # update the board
          board_state[next_move] = 1

      memory_states = np.vstack((memory_states, new_memory))
      if conclude_experience ==1:
          j=1000   # conclude this run
          break



      # machine 2 playes randomly or with minimax ------------------------------
      if rand < 0.3:      # rand
        available_moves = [x  for x in all_moves if x not in whiteList]
        next_move = random.choice(available_moves)
      else:               # minmax
        next_move = machine_turn(n,listOfNodes, whiteList)+1

      whiteList.append(next_move)
      moves_machine2.append(next_move)

      if check_winner(moves_machine2) == True:
          conclude_experience = 1
          reward = r_loss
          new_memory = generate_memory_entry(board_state, next_move-1, reward, conclude_experience,-1 )
      elif len(whiteList) >= 9:
          conclude_experience = 1
          reward = r_draw
          new_memory = generate_memory_entry(board_state, next_move-1, reward, conclude_experience,-1 )


      if conclude_experience ==0:
        reward = r_null
        new_memory = generate_memory_entry(board_state, next_move-1, reward, conclude_experience,-1 )

      # update board state
      board_state[next_move-1] = -1           # new board state

      memory_states = np.vstack((memory_states, new_memory))  # stack experiences
      if conclude_experience ==1:
        j=1000          # conclude this run
        break

  flipped = memory_states[::-1, :].copy()
  return flipped


# ------------------------------------------------------------------------------
# buffer to store played staes
class replayBuffer():
  def __init__(self, size_buffer):
      self.buffer = np.zeros(shape=(size_buffer, 21))
      self.index = 0
      self.size_buffer = size_buffer
      self.is_full = 0

  def clear_all(self):
      for _ in range(self.size_buffer):
        self.buffer[_,:] =  np.array( [0 for _ in range(21)])
      self.index = 0
      self.is_full = 0


  def push(self, entries, dim):  # just because is not a real stack !
      # entries is an array, dim its dimension
      start_index = self.index
      end_index = self.index + dim
      delta = dim
      if end_index > self.size_buffer:
        delta = self.size_buffer - start_index
        end_index = self.size_buffer
        # update
        self.buffer[start_index:end_index, :] = entries[0:delta,:].copy()

        # beginning
        diff = dim - delta   # difference original and already pushed
        self.index = diff
        self.buffer[0:self.index, :] = entries[delta:delta+dim,:].copy()
        self.is_full = 1
      else:
        self.buffer[start_index:end_index, :] = entries[0:dim,:].copy()
        self.index  = end_index


  def random_pop(self, batch_size):
      if self.is_full ==1:
        idx = np.random.choice(self.size_buffer , batch_size, replace=False)
      else:
        idx = np.random.choice( self.index , batch_size, replace=False)

      nbatch = self.buffer[idx].copy()
      return nbatch
 # -----------------------------------------------------------------------------




# model definition
Qmodel = QNN(in_size= 9, hidden_size= 128, out_size= 9 )   #64
#criterion = nn.MSELoss()
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(Qmodel.parameters(), lr=0.0001) #0.001)

# exploration vs esploitation - greedy
greedy_probability_start = 1
greedy_probability_end = 0.1

# for Q learning
discount = 0.95
# initialize breplay bbuffer
# size buffer to store results
size_buffer =  7500
replay_buffer = replayBuffer(size_buffer)



# at each training step, retrieve a batch
batch_size =  100
batch = np.zeros(shape=(batch_size, 21))


# number of episodes
episodes = 7500

greedy_probability = greedy_probability_start
delta_greedy = (greedy_probability_start - greedy_probability_end)/ (episodes-5000)

lst = []
loss_lst = []


# the target net is used in training as substitute of the policy net
target_net = copy.deepcopy(Qmodel)
update_tik = 100      # 150                # update rate


for i in range(episodes):     # training begins

  greedy_probability -= delta_greedy   # update greedy rule
  if greedy_probability < greedy_probability_end:
    greedy_probability = greedy_probability_end

  new_memory_state  = Qtraining( Qmodel, 1,  greedy_probability  ) # play one time
  replay_buffer.push(new_memory_state , 1)

  if replay_buffer.is_full == 1 or replay_buffer.index > 500:     # when the buffer is ready to be used
        batch  = replay_buffer.random_pop(batch_size)             # extract mini-batches

        n_states = len(batch)
        current_states = torch.tensor(batch[:,0:9] +2 , dtype=torch.float32)      #  <--- [-1,0,1] -> [0,1,2]
        next_states = torch.tensor(batch[:,9:18] +2 , dtype=torch.float32)
        actions     = torch.tensor(batch[:, 18], dtype=torch.long).unsqueeze(1)
        rewards     = torch.tensor(batch[:,19], dtype=torch.float32).unsqueeze(1)
        ended       = torch.tensor(batch[:,20], dtype=torch.float32).unsqueeze(1)

        # compute current Q values
        q_values = Qmodel(current_states).gather(1, actions).squeeze()
        # compute future Q values
        with torch.no_grad():
            next_q_values = target_net.forward(next_states).max(1)[0]

        rewards = rewards.squeeze()
        ended = ended.squeeze()
        target_q = rewards + discount * next_q_values * (1 - ended)

        loss =  criterion(q_values, target_q)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i%update_tik == 0:
          target_net.load_state_dict(Qmodel.state_dict())

